# 01_Analysis and Submission 
in this notebook titanic dataset is used and the goal is to predict the survivors of the catastrophic event happend on  1912. The results are later submitted to the Kaggle 


Dataset: https://www.kaggle.com/competitions/titanic/data

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

2024-07-13 23:36:41.227708: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-13 23:36:41.228641: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 23:36:41.233802: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 23:36:41.246387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 23:36:41.266216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

# 01 Analysis of the Data 

In [2]:
current_directory = os.getcwd()
data_directory = current_directory + "/data/titanic"
train_csv = data_directory + "/train.csv"
test_csv = data_directory + "/test.csv"

In [3]:
df_train = pd.read_csv(train_csv)
display(df_train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
print(df_train.info())
print(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

In [5]:
#Cabin has only 204 rows , too many missing rows that's why it is dropped
if "Cabin" in df_train.columns:
    df_train.drop(columns=["Cabin"],inplace=True)
#Age also has missing values. The mean here is 29.6 with std of 14.52
#the missing values are sampled from normal distribution with thos mean and std
normal_ages = np.random.normal(loc=29.69,scale=14.52,size=df_train['Age'].isnull().sum())
df_train.loc[df_train['Age'].isnull(),'Age'] = normal_ages
#Replace missing values in Embarked with most common values - mode
df_train["Embarked"].fillna(df_train["Embarked"].mode()[0],inplace=True)

/tmp/ipykernel_277563/4270252166.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train["Embarked"].fillna(df_train["Embarked"].mode()[0],inplace=True)


In [6]:
#convert catgeroical variables
df_train["Sex"]=LabelEncoder().fit_transform(df_train["Sex"])
#Convert Embarked to one hot encoding - If just convert categories to numerical
#data it can induce that the labels in Embarked are ordered and have different importance
df_train = pd.get_dummies(df_train, columns=["Embarked"],drop_first=True)

In [7]:
#feature and target
X = df_train[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked_Q","Embarked_S"]]
y = df_train["Survived"]

In [8]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state=42)
display(X_train)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
445,1,1,4.000000,0,2,81.8583,False,True
650,3,1,33.507505,0,0,7.8958,False,True
172,3,0,1.000000,1,1,11.1333,False,True
450,2,1,36.000000,1,2,27.7500,False,True
314,2,1,43.000000,1,1,26.2500,False,True
...,...,...,...,...,...,...,...,...
106,3,0,21.000000,0,0,7.6500,False,True
270,1,1,25.760570,0,0,31.0000,False,True
860,3,1,41.000000,2,0,14.1083,False,True
435,1,0,14.000000,1,2,120.0000,False,True


In [9]:
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [10]:
print("X_train shape:", X_train.shape)  # Should print (number_of_samples, number_of_features)
print("y_train shape:", y_train.shape)  # Should print (number_of_samples, )

# Check data types
print("X_train dtype:", X_train.dtype)  # Should ideally be float32 or float64
print("y_train dtype:", y_train.dtype)  # Should ideally be float32 or float64, depending on the task


X_train shape: (623, 8)
y_train shape: (623,)
X_train dtype: float64
y_train dtype: int64


In [22]:
model = Sequential([
    Dense(1024,input_dim=X_train.shape[1],activation="relu"),
    Dense(512,activation="relu"),
    Dense(256,activation="relu"),
    Dense(128,activation="relu"),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),

    Dense(1, activation="sigmoid")
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=['accuracy'])
train = model.fit(X_train,y_train,epochs=300,batch_size=128,validation_data=(X_val,y_val))

Epoch 1/300


2.17.0
